In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [2]:
# load data
df=pd.read_csv('Shuttle_withoutdupl_norm_v01.csv')

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,outlier
0,0.214286,0.607955,0.12766,0.020145,0.207865,0.088428,0.347222,0.746875,0.737113,1
1,0.085714,0.607955,0.12766,0.020145,0.207865,0.088428,0.458333,0.746875,0.716495,1
2,0.200000,0.607955,0.12766,0.020145,0.067416,0.088428,0.361111,0.912500,0.871134,1
3,0.085714,0.602273,0.12766,0.020145,0.207865,0.088428,0.472222,0.746875,0.716495,1
4,0.185714,0.653409,0.12766,0.020145,0.067416,0.088428,0.375000,0.912500,0.865979,1


In [3]:
df.shape

(1013, 10)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Shuttle.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,9]
X = df[:,0:9]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
           # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 15:09:27,746][cascade_classifier.fit_transform] X_groups_train.shape=[(678, 9)],y_train.shape=(678,),X_groups_test.shape=[(335, 9)],y_test.shape=(335,)
[ 2018-04-26 15:09:27,747][cascade_classifier.fit_transform] group_dims=[9]
[ 2018-04-26 15:09:27,749][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:09:27,750][cascade_classifier.fit_transform] group_ends=[9]
[ 2018-04-26 15:09:27,751][cascade_classifier.fit_transform] X_train.shape=(678, 9),X_test.shape=(335, 9)
[ 2018-04-26 15:09:27,753][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(678, 9), X_cur_test.shape=(335, 9)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 7 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[ 2018-04-26 15:09:28,462][kfold_wrapper.log_eval_metrics] Accuracy(la

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:09:37,549][cascade_classifier.transform] X_groups_test.shape=[(335, 9)]
[ 2018-04-26 15:09:37,551][cascade_classifier.transform] group_dims=[9]
[ 2018-04-26 15:09:37,551][cascade_classifier.transform] X_test.shape=(335, 9)
[ 2018-04-26 15:09:37,554][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(335, 9)


Test Accuracy of ExtraTrees = 100.000000 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 15:09:39,286][cascade_classifier.transform] X_groups_test.shape=[(335, 9)]
[ 2018-04-26 15:09:39,288][cascade_classifier.transform] group_dims=[9]
[ 2018-04-26 15:09:39,288][cascade_classifier.transform] X_test.shape=(335, 9)
[ 2018-04-26 15:09:39,291][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(335, 9)


In [11]:
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 15:09:41,472][cascade_classifier.fit_transform] X_groups_train.shape=[(678, 11)],y_train.shape=(678,),X_groups_test.shape=[(335, 11)],y_test.shape=(335,)
[ 2018-04-26 15:09:41,474][cascade_classifier.fit_transform] group_dims=[11]
[ 2018-04-26 15:09:41,476][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:09:41,478][cascade_classifier.fit_transform] group_ends=[11]
[ 2018-04-26 15:09:41,479][cascade_classifier.fit_transform] X_train.shape=(678, 11),X_test.shape=(335, 11)
[ 2018-04-26 15:09:41,481][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(678, 11), X_cur_test.shape=(335, 11)
[ 2018-04-26 15:09:42,324][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:09:43,315][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:09:44,408][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fo

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 15:09:51,839][cascade_classifier.transform] X_groups_test.shape=[(335, 11)]
[ 2018-04-26 15:09:51,841][cascade_classifier.transform] group_dims=[11]
[ 2018-04-26 15:09:51,844][cascade_classifier.transform] X_test.shape=(335, 11)
[ 2018-04-26 15:09:51,846][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(335, 11)


Test Accuracy of ExtraTrees = 100.000000 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print (" Time ",format(round(tt,3))," seconds")

[ 2018-04-26 15:09:54,414][cascade_classifier.fit_transform] X_groups_train.shape=[(678, 11)],y_train.shape=(678,),X_groups_test.shape=[(335, 11)],y_test.shape=(335,)
[ 2018-04-26 15:09:54,417][cascade_classifier.fit_transform] group_dims=[11]
[ 2018-04-26 15:09:54,418][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 15:09:54,419][cascade_classifier.fit_transform] group_ends=[11]
[ 2018-04-26 15:09:54,421][cascade_classifier.fit_transform] X_train.shape=(678, 11),X_test.shape=(335, 11)
[ 2018-04-26 15:09:54,422][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(678, 11), X_cur_test.shape=(335, 11)
[ 2018-04-26 15:09:55,237][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:09:56,065][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:09:57,011][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_fo

('Test Accuracy of ExtraTrees = 100.000000 %', 0, 'fois')
(' Time ', '9.821', ' seconds')


[ 2018-04-26 15:10:07,606][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:10:08,489][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:10:09,472][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-26 15:10:10,502][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:10:11,445][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-26 15:10:12,374][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:10:13,352][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:10:14,155][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=

('Test Accuracy of ExtraTrees = 100.000000 %', 1, 'fois')
(' Time ', '9.444', ' seconds')


[ 2018-04-26 15:10:19,611][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:10:20,620][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:10:21,609][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=100.00%
[ 2018-04-26 15:10:22,743][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:10:23,796][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-26 15:10:24,873][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:10:26,015][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:10:27,037][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=

...

('Test Accuracy of ExtraTrees = 100.000000 %', 17, 'fois')
(' Time ', '9.729', ' seconds')


[ 2018-04-26 15:13:39,930][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:13:41,083][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-26 15:13:42,012][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.53%
[ 2018-04-26 15:13:42,997][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:13:44,051][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.51%
[ 2018-04-26 15:13:45,004][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:13:46,103][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:13:47,182][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=10

('Test Accuracy of ExtraTrees = 99.701493 %', 18, 'fois')
(' Time ', '10.536', ' seconds')


[ 2018-04-26 15:13:52,902][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=100.00%
[ 2018-04-26 15:13:53,773][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.53%
[ 2018-04-26 15:13:54,554][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.53%
[ 2018-04-26 15:13:55,338][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-26 15:13:56,129][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.51%
[ 2018-04-26 15:13:57,110][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=100.00%
[ 2018-04-26 15:13:58,085][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-26 15:13:59,022][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=100

('Test Accuracy of ExtraTrees = 100.000000 %', 19, 'fois')
(' Time ', '9.05', ' seconds')


# ('Test Accuracy of ExtraTrees = 100.000000 %', 0, 'fois')